In [1]:
# imports
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import re

In [2]:
# load data set
df = pd.read_csv("news.csv", encoding='latin-1', header=None) 
df.columns = ['label', 'text']
print(df.head())
print(df.columns)

      label                                               text
0   neutral  According to Gran , the company has no plans t...
1   neutral  Technopolis plans to develop in stages an area...
2  negative  The international electronic industry company ...
3  positive  With the new production plant the company woul...
4  positive  According to the company 's updated strategy f...
Index(['label', 'text'], dtype='object')


In [3]:
# clean text
def clean_text(text):
    text = text.lower() # convert string to lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # remove links
    text = re.sub(r'\@w+|\#','', text)  # remove mentions and hashtags
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # remove numbers and punctuation
    return text
df['text'] = df['text'].astype(str).apply(clean_text) # updates text column to string and cleans

# encode labels
encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['label'])  

print("Label classes:", encoder.classes_)

Label classes: ['negative' 'neutral' 'positive']


In [4]:
# train/tests split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42
)
# here were training it so it can predict what label each text goes under

In [5]:
# create data set class
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from collections import Counter


# splits text at any whitespace
def tokenizer(text):
    return text.split()

# goes through all of the text and yeilds the tokens
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)


def build_vocab(texts, min_freq=1):
    counter = Counter()
    for text in texts:
        counter.update(tokenizer(text))
    # start indices at 2 so we can reserve 0 for PAD and 1 for UNK
    vocab = {word: i+2 for i, (word, freq) in enumerate(counter.items()) if freq >= min_freq}
    vocab["<pad>"] = 0
    vocab["<unk>"] = 1
    return vocab

vocab_dict = build_vocab(train_texts, min_freq=2)  
print("Vocab size:", len(vocab_dict))

def numericalize(text, vocab):
    tokens = tokenizer(text)  # split into words
    return [vocab.get(token, vocab["<unk>"]) for token in tokens]



Vocab size: 4039


In [6]:
# import numpy as np
# import torch

# def load_glove_embeddings(glove_file_path, vocab, embedding_dim = 50):
#     embeddings_index = {}
#     with open(glove_file_path, encoding="utf8") as f:
#         for line in f:
#             values = line.split() #split at every space
#             word = values[0] #the word is the first thing in each line
#             vector = np.asarray(values[1:], dtype="float32") #convert rest of inputs in line to numpy array
#             embeddings_index[word] = vector
    
#     embedding_matrix = np.zeros((max(vocab.values()) + 1, embedding_dim)) #nump array filled with 0's initially and has a dimention of 50 for each word
#     # function adds word's vector into our matrix
#     for word, idx in vocab.items():
#         vector = embeddings_index.get(word)
#         if vector is not None:
#             embedding_matrix[idx] = vector
#         else:
#             embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))

#     return torch.tensor(embedding_matrix, dtype=torch.float32)

# embedding_matrix = load_glove_embeddings("glove.6B.50d.txt", vocab_dict, embedding_dim=50)




In [7]:

class NewsDataset(Dataset):
    def __init__(self, texts, labels, vocab):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab

    # number of samples in dataset
    def __len__(self):
        return len(self.texts)

    # call when we want only one sample
    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        tokens = torch.tensor(numericalize(text, self.vocab), dtype=torch.long) #self.vocab is for mapping words to numbers
        return tokens, torch.tensor(label, dtype=torch.long) #returns tokens and labels
    
# make data into dataset object
train_dataset = NewsDataset(train_texts, train_labels, vocab_dict)
test_dataset = NewsDataset(test_texts, test_labels, vocab_dict)

def collate_batch(batch):
    texts, labels = zip(*batch)
    texts = pad_sequence(texts, batch_first=True, padding_value=0) #makes all lists same length by adding zeros at end
    labels = torch.tensor(labels, dtype=torch.long)
    return texts, labels

# train 32 healines per batch
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_batch) # reshuffle every epoch
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=collate_batch)

print("Train batches:", len(train_loader))
print("Test batches:", len(test_loader))





Train batches: 122
Test batches: 31


In [8]:
import torch.nn as nn

class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)  #turns vocab id into dense vector
        #self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.fc = nn.Linear(embed_dim, num_classes) # input is average embedding of a sentence and output is num_classes(2)cwhich is buy or sell
    
    def forward(self, x):
        # x = (batch size, seq len)
        embedded = self.embedding(x) # gives us original output and embed_dim
        pooling = embedded.mean(dim = 1) # average embedding across all words in teh sentence, used to get one vec per sentence
        output = self.fc(pooling) # passes vector into classifier
        return output

In [9]:
#initialize model, loss, optimizer
vocab_size = max(vocab_dict.values()) + 1 # amount of unique words
embed_dim = 50 # size of each word vector, the larger the number the more expressive the word is
num_classes = len(set(train_labels)) # number of classes -> 2(buy/sell)
model = TextClassifier(vocab_size, embed_dim, num_classes)
criterion = nn.CrossEntropyLoss() # diff between prediction and target
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001) # optimizer that updates weights using gradients


In [10]:
# model training loop
def train_model(self, train_loader, criterion, optimizer, epochs = 5):
    model.train() # put in training mode
    for epoch in range(epochs):
        total_loss = 0
        for texts, label in train_loader:
            optimizer.zero_grad() # reset gradients
            outputs = model(texts) # forward pass
            loss = criterion(outputs, label) # calculate error
            loss.backward() # back propagation
            optimizer.step() # update weights
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

In [11]:
def evaluate_model(self, test_loader):
    model.eval() # put in evaluation mode
    correct, total = 0, 0
    with torch.no_grad(): # gradients not needed in eval mode
        for texts, labels in test_loader:
            outputs = model(texts) # forward pass
            _, predicted = torch.max(outputs, 1) # get prediction index 0 = sell 1 = buy
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

In [19]:
# Train the model
train_model(model, train_loader, criterion, optimizer, epochs=1)

torch.save(model.state_dict(), "sentiment_model.pth")

# Evaluate on test set
evaluate_model(model, test_loader)


Epoch 1/1, Loss: 27.8095
Test Accuracy: 77.01%


In [13]:
# use model
import torch.nn.functional as F
def prediction(text, model, vocab, max_len = 50):
    model.eval()
    with torch.no_grad():
        # tokenize
        tokens = text.lower().split()
        token_ids = [vocab_dict.get(word, vocab["<unk>"]) for word in tokens]

        # pad
        if len(tokens) < max_len:
            token_ids += [vocab["<pad>"]] * (max_len - len(token_ids))
        else:
            token_ids = token_ids[:max_len]

        input_tensor = torch.tensor([token_ids])
        output = model(input_tensor)

        probs = F.softmax(output, dim=1) # rescales input and has it sum to 1, probability distribution
        predicted_class = torch.argmax(probs, dim=1).item() # picks class with highest probability
        return predicted_class, probs.numpy()

In [ ]:


# Recreate vocab + model
model = TextClassifier(vocab_size, embed_dim, num_classes)
model.load_state_dict(torch.load("sentiment_model.pth"))
model.eval()

# Then predict
sentiment = prediction("Tesla stock surges after earnings", model, vocab_dict)
print(sentiment)

(2, array([[0.26972422, 0.33207425, 0.3982015 ]], dtype=float32))


In [ ]:
sample_text = "Stock prices soar after company reports record earnings"
pred_class, pred_probs = prediction(sample_text, model, vocab_dict)

print("Prediction:", pred_class)
print("Probabilities:", pred_probs)
example_texts = [
    "Stock prices soared after the company announced record profits",
    "The market crashed due to unexpected inflation data",
    "Investors are optimistic about new technology developments",
    "Stock prices soar after company reports record earnings",
    
]

for txt in example_texts:
    pred_class, probs = prediction(txt, model, vocab_dict)
    print(f"Text: {txt}")
    print(f"Predicted Class: {pred_class}, Probabilities: {probs}\n")


Prediction: 2
Probabilities: [[0.26972422 0.33207425 0.3982015 ]]
Text: Stock prices soared after the company announced record profits
Predicted Class: 1, Probabilities: [[0.18211912 0.5038501  0.31403086]]

Text: The market crashed due to unexpected inflation data
Predicted Class: 1, Probabilities: [[0.10302711 0.8363166  0.06065637]]

Text: Investors are optimistic about new technology developments
Predicted Class: 1, Probabilities: [[0.0652108  0.70962    0.22516921]]

Text: Stock prices soar after company reports record earnings
Predicted Class: 2, Probabilities: [[0.26972422 0.33207425 0.3982015 ]]



In [ ]:
import torch, torch.nn.functional as F, numpy as np

def debug_predict(text, model, vocab, label_map, max_len=50):
    model.eval()
    tokens = text.lower().split()
    token_ids = [vocab.get(w, vocab.get("<unk>", 1)) for w in tokens]
    print("Text:", text)
    print("Tokens:", tokens)
    print("Token IDs:", token_ids)
    unk_count = sum(1 for w in tokens if w not in vocab)
    print(f"UNKs: {unk_count}/{len(tokens)} ({unk_count/len(tokens):.2%})")

    # pad/truncate
    if len(token_ids) < max_len:
        token_ids += [vocab.get("<pad>", 0)] * (max_len - len(token_ids))
    else:
        token_ids = token_ids[:max_len]

    with torch.no_grad():
        out = model(torch.tensor([token_ids]))
        probs = F.softmax(out, dim=1).cpu().numpy()[0]
        pred = int(np.argmax(probs))

    print("Predicted index:", pred, "Label:", label_map[pred])
    print("Probabilities:", {label_map[i]: float(probs[i]) for i in range(len(probs))})
    return pred, probs

# Use it:
#debug_predict("The market crashed due to unexpected inflation data", model, vocab_dict, label_map)


NameError: name 'label_map' is not defined